In [3]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime
from key import BASE_URL

In [ ]:

print(requests.get(f'{BASE_URL}/').text)

Hello Server



In [48]:
with open('./stock_name.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [4]:
def norm(column):
    norm_value = np.linalg.norm(column)
    if norm_value == 0:
        return column  # norm が 0 の場合、そのままの値を返す
    else:
        return column / norm_value, norm_value

In [50]:
for category in data:
    for company in data[category]:
        stock_code = company['symbol']
        print(stock_code)

A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ATO
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BG
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBRE
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CRL
CRM
CRWD
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQT
ES
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOX
FOXA
FSLR
FTNT
FTV
GD
GDDY
GE
GEHC
GEN
GEV
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
INCY
INTC
INTU
IP
IPG
IQV
IR
ISRG
IT
ITW
I

In [45]:
for category in data:
    for company in data[category]:
        stock_code = company['symbol']
        print(stock_code)

        df = pd.DataFrame(requests.get(f'{BASE_URL}/ml_data/{stock_code}').json())
        print(df.info())
        
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        
        path = f'./ml_data/{stock_code}.csv'
        df.to_csv(f'./ml_data/{stock_code}_pre.csv', index=False, encoding='utf-8')

        exclude_columns = ['date', 'time']
        columns = [col for col in df.columns if col not in exclude_columns]
        print(columns)
        
        # 各列を個別に正規化して、辞書に格納
        ny_dow = norm(df['NY_Dow'])[0]
        sp_500 = norm(df['SP_500'])[0]
        content_concern = df['content_concern']
        content_despair = df['content_despair']
        content_excitement = df['content_excitement']
        content_optimism = df['content_optimism']
        content_stability = df['content_stability']
        headline_concern = df['headline_concern']
        headline_despair = df['headline_despair']
        headline_excitement = df['headline_excitement']
        headline_optimism = df['headline_optimism']
        headline_stability = df['headline_stability']
        value = norm(df['value'])[0]
        vix = norm(df['vix'])[0]

        # 正規化されたデータを辞書にまとめる
        data_dict = {
            'NY_Dow': ny_dow,
            'SP_500': sp_500,
            'content_concern': content_concern,
            'content_despair': content_despair,
            'content_excitement': content_excitement,
            'content_optimism': content_optimism,
            'content_stability': content_stability,
            'headline_concern': headline_concern,
            'headline_despair': headline_despair,
            'headline_excitement': headline_excitement,
            'headline_optimism': headline_optimism,
            'headline_stability': headline_stability,
            'vix': vix,
            'value': value
        }

        # 辞書をデータフレームに変換
        data = pd.DataFrame(data_dict)

        # 同じ日の要素ごとに和を取る列
        sum_columns = ['content_concern', 'content_despair', 'content_excitement', 
                       'content_optimism', 'content_stability', 'headline_concern',
                       'headline_despair', 'headline_excitement', 'headline_optimism',
                       'headline_stability']
        
        # 和を取る列は日毎に集約
        summed_data = data[sum_columns].resample('D').sum()

        # それ以外の列は最大値を取る
        max_columns = [col for col in data.columns if col not in sum_columns]
        max_data = data[max_columns].resample('D').max()

        # 両方を結合
        data = pd.concat([summed_data, max_data], axis=1)
        
        data_dict = {
                'NY_Dow': data['NY_Dow'],
                'SP_500': data['SP_500'],
                'content_concern': norm(data['content_concern'])[0],
                'content_despair': norm(data['content_despair'])[0],
                'content_excitement': norm(data['content_excitement'])[0],
                'content_optimism': norm(data['content_optimism'])[0],
                'content_stability': norm(data['content_stability'])[0],
                'headline_concern': norm(data['headline_concern'])[0],
                'headline_despair': norm(data['headline_despair'])[0],
                'headline_excitement': norm(data['headline_excitement'])[0],
                'headline_optimism': norm(data['headline_optimism'])[0],
                'headline_stability': norm(data['headline_stability'])[0],
                'vix': data['vix'],
                'value': data['value']
        }
        
        data = pd.DataFrame(data_dict)
        
        data.to_csv(path, index=False, encoding='utf-8')

A
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NY_Dow               2999 non-null   float64
 1   SP_500               2999 non-null   float64
 2   content_concern      2999 non-null   float64
 3   content_despair      2999 non-null   float64
 4   content_excitement   2999 non-null   float64
 5   content_optimism     2999 non-null   float64
 6   content_stability    2999 non-null   float64
 7   date                 2999 non-null   object 
 8   headline_concern     2999 non-null   float64
 9   headline_despair     2999 non-null   float64
 10  headline_excitement  2999 non-null   float64
 11  headline_optimism    2999 non-null   float64
 12  headline_stability   2999 non-null   float64
 13  time                 2999 non-null   object 
 14  value                2999 non-null   float64
 15  vix                  2999 non-null  

KeyboardInterrupt: 